In [1]:
import pandas as pd
import nltk
import gensim
from gensim.corpora import Dictionary
from gensim.models import LdaModel
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [2]:
## read data

df = pd.read_csv("~/Documents/all_filings_and_sections.csv")

In [3]:

df.head()


,Unnamed: 0,ticker,companyName,formType,description,filedAt,linkToFilingDetails,Section1,Section1A,Section7
0,0,AIZ,"ASSURANT, INC.",10-K,Form 10-K - Annual report [Section 13 and 15(d...,2023-02-17T16:12:13-05:00,https://www.sec.gov/Archives/edgar/data/126723...,"Item 1. Business \n\nAssurant, Inc. was incor...",Item 1A. Risk Factors \n\nCertain factors may...,Item 7. Management&#8217;s Discussion and Ana...
1,1,AIZ,"ASSURANT, INC.",10-K,Form 10-K - Annual report [Section 13 and 15(d...,2022-02-22T16:24:39-05:00,https://www.sec.gov/Archives/edgar/data/126723...,"Item 1. Business \n\nAssurant, Inc. was incor...",Item 1A. Risk Factors \n\nCertain factors may...,Item 7. Management&#8217;s Discussion and Ana...
2,2,AIZ,"ASSURANT, INC.",10-K,Form 10-K - Annual report [Section 13 and 15(d...,2021-02-19T16:44:57-05:00,https://www.sec.gov/Archives/edgar/data/126723...,"Item 1. Business \n\nAssurant, Inc. was incor...",Item 1A. Risk Factors \n\nCertain factors may...,Item 7. Management&#8217;s Discussion and Ana...
3,3,AIZ,"ASSURANT, INC.",10-K,Form 10-K - Annual report [Section 13 and 15(d...,2020-02-19T17:13:43-05:00,https://www.sec.gov/ix?doc=/Archives/edgar/dat...,"Item 1. Business \n\nAssurant, Inc. was incor...",Item 1A. Risk Factors \n\nCertain factors may...,Item 7. Management&#8217;s Discussion and Ana...
4,4,AIZ,ASSURANT INC,10-K,Form 10-K - Annual report [Section 13 and 15(d...,2019-02-22T16:48:45-05:00,https://www.sec.gov/ix?doc=/Archives/edgar/dat...,"Item 1. Business \n\nAssurant, Inc. was incor...",Item 1A. Risk Factors \n\nCertain factors may...,Item 7. Management&#8217;s Discussion and Ana...


In [6]:
df['linkToFilingDetails'][2000]

'https://www.sec.gov/Archives/edgar/data/920760/000162828023001606/len-20221130.htm'

### QUESTION 1: *What are the common topics mentioned in the SEC filings?

In [ ]:
# sum(df.Section1.isna())
df = df.dropna()

In [108]:
# Load the generated dataset (Replace with your dataset loading code if necessary)
# data = pd.read_csv('sample_data.csv')
data = df

# Download NLTK data
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')


[nltk_data] Downloading package punkt to /home/pengfei/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/pengfei/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/pengfei/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [103]:


# df['Section1'] = df['Section1'].fillna('')

In [104]:
sum(df.Section1.isna())


0

In [6]:
# Prepare stop words
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess(text):
    tokens = [word for sent in sent_tokenize(text) for word in word_tokenize(sent)]
    tokens = [word.lower() for word in tokens if word.isalpha()]
    tokens = [word for word in tokens if word not in stop_words]
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    return tokens

In [109]:
data["Section1"] = data["Section1"].apply(lambda x: [x])
data["Section1A"] = data["Section1A"].apply(lambda x: [x])
data["Section7"] = data["Section7"].apply(lambda x: [x])

In [110]:
data.head()

,Unnamed: 0,ticker,companyName,formType,description,filedAt,linkToFilingDetails,Section1,Section1A,Section7
0,0,AIZ,"ASSURANT, INC.",10-K,Form 10-K - Annual report [Section 13 and 15(d...,2023-02-17T16:12:13-05:00,https://www.sec.gov/Archives/edgar/data/126723...,"[ Item 1. Business \n\nAssurant, Inc. was inco...",[ Item 1A. Risk Factors \n\nCertain factors ma...,[ Item 7. Management&#8217;s Discussion and An...
1,1,AIZ,"ASSURANT, INC.",10-K,Form 10-K - Annual report [Section 13 and 15(d...,2022-02-22T16:24:39-05:00,https://www.sec.gov/Archives/edgar/data/126723...,"[ Item 1. Business \n\nAssurant, Inc. was inco...",[ Item 1A. Risk Factors \n\nCertain factors ma...,[ Item 7. Management&#8217;s Discussion and An...
2,2,AIZ,"ASSURANT, INC.",10-K,Form 10-K - Annual report [Section 13 and 15(d...,2021-02-19T16:44:57-05:00,https://www.sec.gov/Archives/edgar/data/126723...,"[ Item 1. Business \n\nAssurant, Inc. was inco...",[ Item 1A. Risk Factors \n\nCertain factors ma...,[ Item 7. Management&#8217;s Discussion and An...
3,3,AIZ,"ASSURANT, INC.",10-K,Form 10-K - Annual report [Section 13 and 15(d...,2020-02-19T17:13:43-05:00,https://www.sec.gov/ix?doc=/Archives/edgar/dat...,"[ Item 1. Business \n\nAssurant, Inc. was inco...",[ Item 1A. Risk Factors \n\nCertain factors ma...,[ Item 7. Management&#8217;s Discussion and An...
4,4,AIZ,ASSURANT INC,10-K,Form 10-K - Annual report [Section 13 and 15(d...,2019-02-22T16:48:45-05:00,https://www.sec.gov/ix?doc=/Archives/edgar/dat...,"[ Item 1. Business \n\nAssurant, Inc. was inco...",[ Item 1A. Risk Factors \n\nCertain factors ma...,[ Item 7. Management&#8217;s Discussion and An...


In [111]:
# Prepare dataset for LDA analysis
all_texts = data["Section1"].tolist() + data["Section1A"].tolist() + data["Section7"].tolist()
all_texts = [" ".join(section) for section in all_texts]
processed_texts = [preprocess(text) for text in all_texts]


In [112]:
# Create Gensim dictionary, filter extremes, and convert to a bag-of-words corpus
dictionary = Dictionary(processed_texts)
dictionary.filter_extremes(no_below=10, no_above=0.5, keep_n=100000)
corpus = [dictionary.doc2bow(text) for text in processed_texts]

In [113]:
# Parameters for LDA
num_topics = 5
random_state = 100

In [114]:
# Train LDA model
lda = LdaModel(
    corpus,
    num_topics=num_topics,
    id2word=dictionary,
    random_state=random_state,
    update_every=1,
    passes=10,
    alpha='auto',
    per_word_topics=True
)

In [115]:
# Print topics
topics = lda.print_topics(num_words=5)

for topic in topics:
    print(topic)

(0, '0.011*"loan" + 0.010*"client" + 0.007*"bank" + 0.005*"deposit" + 0.005*"reputation"')
(1, '0.015*"store" + 0.007*"real" + 0.007*"estate" + 0.007*"home" + 0.006*"tenant"')
(2, '0.029*"gas" + 0.016*"utility" + 0.013*"electric" + 0.012*"oil" + 0.010*"fuel"')
(3, '0.008*"care" + 0.007*"patient" + 0.006*"patent" + 0.006*"clinical" + 0.006*"device"')
(4, '0.012*"partially" + 0.011*"adjusted" + 0.009*"loan" + 0.009*"table" + 0.008*"intangible"')


In [118]:
lda.save('lda_model.gensim')
dictionary.save('dictionary.gensim')

print("LDA model and dictionary have been trained and saved.")

LDA model and dictionary have been trained and saved.


### QUESTION 2: What are the emerging topics or financial themes over the last 5 years?

In [116]:
# Add a "year" column for grouping

data['filedAt'] = data['filedAt'].astype(str)
data['year'] = data['filedAt'].apply(lambda x: x[:4]).astype(int)

In [117]:
data.head()

,Unnamed: 0,ticker,companyName,formType,description,filedAt,linkToFilingDetails,Section1,Section1A,Section7,year
0,0,AIZ,"ASSURANT, INC.",10-K,Form 10-K - Annual report [Section 13 and 15(d...,2023-02-17T16:12:13-05:00,https://www.sec.gov/Archives/edgar/data/126723...,"[ Item 1. Business \n\nAssurant, Inc. was inco...",[ Item 1A. Risk Factors \n\nCertain factors ma...,[ Item 7. Management&#8217;s Discussion and An...,2023
1,1,AIZ,"ASSURANT, INC.",10-K,Form 10-K - Annual report [Section 13 and 15(d...,2022-02-22T16:24:39-05:00,https://www.sec.gov/Archives/edgar/data/126723...,"[ Item 1. Business \n\nAssurant, Inc. was inco...",[ Item 1A. Risk Factors \n\nCertain factors ma...,[ Item 7. Management&#8217;s Discussion and An...,2022
2,2,AIZ,"ASSURANT, INC.",10-K,Form 10-K - Annual report [Section 13 and 15(d...,2021-02-19T16:44:57-05:00,https://www.sec.gov/Archives/edgar/data/126723...,"[ Item 1. Business \n\nAssurant, Inc. was inco...",[ Item 1A. Risk Factors \n\nCertain factors ma...,[ Item 7. Management&#8217;s Discussion and An...,2021
3,3,AIZ,"ASSURANT, INC.",10-K,Form 10-K - Annual report [Section 13 and 15(d...,2020-02-19T17:13:43-05:00,https://www.sec.gov/ix?doc=/Archives/edgar/dat...,"[ Item 1. Business \n\nAssurant, Inc. was inco...",[ Item 1A. Risk Factors \n\nCertain factors ma...,[ Item 7. Management&#8217;s Discussion and An...,2020
4,4,AIZ,ASSURANT INC,10-K,Form 10-K - Annual report [Section 13 and 15(d...,2019-02-22T16:48:45-05:00,https://www.sec.gov/ix?doc=/Archives/edgar/dat...,"[ Item 1. Business \n\nAssurant, Inc. was inco...",[ Item 1A. Risk Factors \n\nCertain factors ma...,[ Item 7. Management&#8217;s Discussion and An...,2019


In [124]:

# LDA analysis for a given year
def analyze_year(year, num_topics=5, random_state=100):
    year_data = data[data["year"] == year]
    all_texts = year_data["Section1"].tolist() + year_data["Section1A"].tolist() + year_data["Section7"].tolist()
    all_texts = [" ".join(section) for section in all_texts]
    processed_texts = [preprocess(text) for text in all_texts]
#     print(processed_texts)
    dictionary = Dictionary(processed_texts)
    dictionary.filter_extremes(no_below=10, no_above=0.5, keep_n=100000)
    corpus = [dictionary.doc2bow(text) for text in processed_texts]

    lda = LdaModel(
        corpus,
        num_topics=num_topics,
        id2word=dictionary,
        random_state=random_state,
        update_every=1,
        passes=10,
        alpha="auto",
        per_word_topics=True,
    )

    topics = lda.print_topics(num_words=5)
    return topics, processed_texts



In [125]:
from collections import defaultdict

In [126]:
def get_word_frequencies(words, processed_texts):
    word_frequencies = defaultdict(int)
    for word in words:
        for text in processed_texts:
            word_frequencies[word] += text.count(word)
    return word_frequencies

unique_years = sorted(data["year"].unique())
all_topics = []
all_processed_texts = []

In [121]:
data[data['year'] == 2020]

,Unnamed: 0,ticker,companyName,formType,description,filedAt,linkToFilingDetails,Section1,Section1A,Section7,year
3,3,AIZ,"ASSURANT, INC.",10-K,Form 10-K - Annual report [Section 13 and 15(d...,2020-02-19T17:13:43-05:00,https://www.sec.gov/ix?doc=/Archives/edgar/dat...,"[ Item 1. Business \n\nAssurant, Inc. was inco...",[ Item 1A. Risk Factors \n\nCertain factors ma...,[ Item 7. Management&#8217;s Discussion and An...,2020
8,8,CSGP,"COSTAR GROUP, INC.",10-K,Form 10-K - Annual report [Section 13 and 15(d...,2020-02-26T16:15:47-05:00,https://www.sec.gov/ix?doc=/Archives/edgar/dat...,[ Item 1. \n\nBusiness \n\n##TABLE_END In this...,[ Item 1A. \n\nRisk Factors \n\n##TABLE_END \n...,[ Item 7. \n\nManagement&#8217;s Discussion an...,2020
12,12,J,JACOBS ENGINEERING GROUP INC /DE/,10-K,Form 10-K - Annual report [Section 13 and 15(d...,2020-11-24T07:13:57-05:00,https://www.sec.gov/Archives/edgar/data/52988/...,"[ Item 1. BUSINESS \n\nAt Jacobs, we&#8217;re ...",[ Item 1 A. RISK FACTORS \n\nWe operate in a c...,[ Item 7. MANAGEMENT'S DISCUSSION AND ANALYSIS...,2020
17,17,KR,KROGER CO,10-K,Form 10-K - Annual report [Section 13 and 15(d...,2020-04-01T09:01:18-04:00,https://www.sec.gov/ix?doc=/Archives/edgar/dat...,[ ITEM 1. \n\nBUSINESS. \n\n##TABLE_END &#8203...,[ ITEM 1A. \n\nRISK FACTORS. \n\n##TABLE_END &...,[ ITEM 7. \n\nMANAGEMENT&#8217;S DISCUSSION AN...,2020
22,22,AON,Aon plc,10-K,Form 10-K - Annual report [Section 13 and 15(d...,2020-02-14T16:40:54-05:00,https://www.sec.gov/ix?doc=/Archives/edgar/dat...,[ Item 1. Business \n\nOVERVIEW \n\nAon plc (w...,[ Item 1A. Risk Factors \n\nThe risk factors s...,[ Item 7. Management&#8217;s Discussion and An...,2020
...,...,...,...,...,...,...,...,...,...,...,...
2394,2394,UDR,"UDR, Inc.",10-K,Form 10-K - Annual report [Section 13 and 15(d...,2020-02-18T16:58:55-05:00,https://www.sec.gov/ix?doc=/Archives/edgar/dat...,[ Item 1. BUSINESS\n\nGeneral\n\nUDR is a self...,[ Item 1A.\n\nRISK FACTORS\n\n##TABLE_END Ther...,[ Item 7.\n\nMANAGEMENT&#8217;S DISCUSSION AND...,2020
2399,2399,PWR,"QUANTA SERVICES, INC.",10-K,Form 10-K - Annual report [Section 13 and 15(d...,2020-02-28T17:10:05-05:00,https://www.sec.gov/ix?doc=/Archives/edgar/dat...,[ ITEM 1. Business \n\nGeneral \n\nQuanta Serv...,[ ITEM 1A. Risk Factors \n\nOur business is su...,[ ITEM 7. \n\nManagement&#8217;s Discussion an...,2020
2404,2404,EXPD,EXPEDITORS INTERNATIONAL OF WASHINGTON INC,10-K,Form 10-K - Annual report [Section 13 and 15(d...,2020-02-21T15:01:31-05:00,https://www.sec.gov/ix?doc=/Archives/edgar/dat...,[ ITEM 1&#8212;BUSINESS\n\nOverview\n\nExpedit...,[ ITEM 1A &#8211; RISK FACTORS\n\n&#160;\n\n##...,[ ITEM 7 &#8212; MANAGEMENT&#8217;S DISCUSSION...,2020
2414,2414,HWM,Arconic Inc.,10-K,Form 10-K - Annual report [Section 13 and 15(d...,2020-02-26T17:48:56-05:00,https://www.sec.gov/ix?doc=/Archives/edgar/dat...,[ Item 1. Business. \n\nGeneral \n\nArconic In...,[ Item 1A. Risk Factors. \n\nArconic&#8217;s b...,[ Item 7. Management&#8217;s Discussion and An...,2020


In [142]:
data['linkToFilingDetails'][0]

'https://www.sec.gov/Archives/edgar/data/1267238/000126723823000007/aiz-20221231.htm'

In [127]:
# Group documents by year and analyze each year
unique_years = sorted(data['year'].unique())

for year in unique_years:
    topics, processed_texts = analyze_year(year, num_topics=5, random_state=100)
    all_topics.extend(topics)
    all_processed_texts.extend(processed_texts)

In [129]:
# Extract topic words
topic_words = set()
for topic in all_topics:
    words = topic[1].split(" + ")
    for word in words:
        topic_words.add(word.split("*")[1][1:-1])

# Get word frequencies over time
word_frequencies = get_word_frequencies(topic_words, all_processed_texts)

In [130]:
# Evaluate emerging themes
emerging_themes = []
ratios = [(word, word_frequencies[word] / len(all_processed_texts)) for word in topic_words]
ratios.sort(key=lambda x: x[1], reverse=True)


In [131]:
ratios[:10]

[('fiscal', 9.715752461322081),
 ('gas', 8.048241912798876),
 ('loan', 7.660056258790436),
 ('client', 5.764838255977496),
 ('table', 5.106469760900141),
 ('partially', 5.09535864978903),
 ('manufacturing', 5.056118143459916),
 ('adjusted', 5.008860759493671),
 ('care', 4.1884669479606185),
 ('bank', 4.075668073136428)]

In [132]:

for word, ratio in ratios:
    if ratio > 4:  # Adjust threshold according to the analysis needs
        emerging_themes.append(word)

print("Emerging themes:", emerging_themes)

Emerging themes: ['fiscal', 'gas', 'loan', 'client', 'table', 'partially', 'manufacturing', 'adjusted', 'care', 'bank']


### QUESTION 3:  **What are the declining themes?*

In [133]:
ratios[-10:]

[('drug', 2.4773558368495077),
 ('emission', 2.4362869198312236),
 ('harm', 2.3756680731364277),
 ('privacy', 2.2677918424753867),
 ('breach', 2.1912798874824193),
 ('reinsurance', 2.0924050632911393),
 ('mortgage', 2.080872011251758),
 ('fail', 1.9756680731364276),
 ('fda', 1.8825597749648382),
 ('tenant', 1.7047819971870606)]

In [134]:

declining_themes = []

for word, ratio in ratios:
    if ratio < 2.5:  # Adjust threshold according to the analysis needs
        declining_themes.append(word)

print("Declining themes:", declining_themes)

Declining themes: ['drug', 'emission', 'harm', 'privacy', 'breach', 'reinsurance', 'mortgage', 'fail', 'fda', 'tenant']


### question 4  **What is the sentiment over those themes?

In [135]:
from textblob import TextBlob

In [136]:
# Preprocess the text to obtain the tokens for each section
data['tokens'] = data['Section1'] + data['Section1A'] + data['Section7']
data['tokens'] = data['tokens'].apply(lambda x: preprocess(x[0]))


In [137]:

# Check if a section contains any of the declining themes
data['contains_declining_theme'] = data['tokens'].apply(lambda x: any(theme in x for theme in declining_themes))


In [138]:

# Compute sentiment polarity and subjectivity for each section using TextBlob
data['sentiment_polarity'] = data['tokens'].apply(lambda x: TextBlob(' '.join(x)).sentiment.polarity)
data['sentiment_subjectivity'] = data['tokens'].apply(lambda x: TextBlob(' '.join(x)).sentiment.subjectivity)

In [139]:
# Filter the sections that contain declining themes
declining_theme_sections = data[data['contains_declining_theme']]

In [140]:

# Calculate average sentiment polarity and subjectivity
average_sentiment_polarity = declining_theme_sections['sentiment_polarity'].mean()
average_sentiment_subjectivity = declining_theme_sections['sentiment_subjectivity'].mean()

In [141]:
print(f'Average sentiment polarity for sections containing declining themes: {average_sentiment_polarity:.2f}')
print(f'Average sentiment subjectivity for sections containing declining themes: {average_sentiment_subjectivity:.2f}')

Average sentiment polarity for sections containing declining themes: 0.07
Average sentiment subjectivity for sections containing declining themes: 0.36
